This script will parse the report returned to you, once you choose 'Finalize All' in the Finalize area of the NACC portal.

This Finalization report which will be parsed has all of the error information related to your stuck packets -- visit numbers, forms, etc, along with the actual error text. 

(Keep in mind, however, this won't get one particular class of errors. The final error check by the NACC, called the Cross Visit error check, is done by the NACC in a separate sequential step, the results of which come in a second email delivered after Finalize All is submitted. So, this script will retrieve a center's Within Form and Cross Form errors only. Fortunately, this is usually the bulk of the errors.) 

To create a csv of the MAC's errors:
 -  first log on to the NACC portal, and navigate to the Error Check sidebar, within the FTD UDS Submission area.
 -  Select "All Packets" and set 'Select Number of Errors to Display:' to 'All Errors'. No need to create an excel file.
 -  Choose the 'Batch Job' option, enter your email, submit and wait for the first response email.
 
When you get the email back (titled 'UDS Data Final Checks'), click on the attached link to open in your default browser.

Find and right click the 'Print File' link on the webpage, and save as 'ferrorNN.pff'.
Now, open in Sublime and save as a csv (this is the least painful way to retain formatting). Make sure the error report is in the same directory that this script lives in.

Finally, run this script and find the outputted .csv


In [9]:
import csv
import pandas as pd
import numpy as np
import mysql.connector
import datetime
import matplotlib.pyplot as plt

In [10]:
## Read file; perform basic cleaning
# Remember, you must get a file name 'ferrorNN.pff' after finalizing (check the first email and open in browser: right click the link and save this locally)
# Open in Sublime text editor or whatever editor you prefer, and save as a .csv. 
# Then set the name here so that you can open it.

df = []
with open('ferrorXX.csv', 'U') as f:
    reader = csv.reader(f)
    for row in reader:
        df.append(row)                      # Each row is a list

df = [alist for alist in df if alist != []] # Remove empty lists (these are corresponding to blank lines in the ferror.csv)

df1 = [[' '.join(alist)] for alist in df]           # For each list, rejoin the strings inside if there are more than one. This keeps the error messages together in one string, which may have been split on commas when the file was read in (as some error messages contain commas).
df2 = [item for sublist in df1 for item in sublist] # Flatten our list of lists; now we have one list, where each string inside is one complete row just as you would find on the csv 
df3 = [astring.strip() for astring in df2]          # Strip whitespace from the beginning and end of each string / error message
df4 = [astring.strip().split() for astring in df3]
df5 = [' '.join(alist) for alist in df4] 

# At this point, we should have one list of strings, where each string is a line from the ferrror.csv file, with excess whitespace removed and commas within the error removed.

df5[:5]

# Continue with cleaning and prep within a Pandas DataFrame
df = pd.DataFrame(df5)

mask = df.applymap(lambda x: x is None) # replace all None with ''
cols = df.columns[(mask).any()]
for col in df[cols]:
    df.loc[mask[col], col] = ''
    
df = pd.DataFrame(df.iloc[1:].apply(lambda x: ''.join(x), axis=1)) # This collapses all columns into one column -- later we deconstruct this single column into all others
df.columns = ['data']                                            # Name this single column 'data'

# Remove header: this is the first block on the csv that shows site number and the (erroneous, by the way) report of numbers of packets in the working database.
# The errors we want start after the line "All packets processed!"
first_row_idx = df[df.data.str.match('All packets processed!')].index[0] +1
df = df.loc[first_row_idx:, ]

df['data'] = df['data'].map(str.strip) # We want to make sure there is only one whitespace between every token

# Remove footer, which has no useful information for the purpose of creating this report:
last_row_idx = df[df.data.str.match('The following Packets have not been moved to the current database')].index[0] -1
df = df.loc[:last_row_idx, ]

# Remove all empty lines and lines that have only --------------------------------------------------------------------------------------------------
df = df[df.data != '--------------------------------------------------------------------------------------------------']
df = df[df.data != '']
df = df[df.data !=
        'Item       Data Element E-#  Error                                                      Value']
df = df[df.data != 'Item       Data Element E-#  Error                               Var Comp      Value']
df = df[df.data != 'Item       Data Element E-#  Error                               Value         Cross with ']
df = df[df.data != 'Item       Data Element  E-# Error                                   Var Comp  Value']
df = df[df.data != 'All packets processed!']

##### we are ready to start creating the full dataframe by using a strategy of creating fresh columns and copying 
# the appropriate info over from 'data' column, then ffilling ('fill forward') down.
# This is the general strategy: use matching to find the information we want, then create a new column out of it.
# We take advantage of the fact that the NACC's error messages have (for the most part) fixed character lengths. 
# This means the PIDN, message, type, variable, etc, all occupy a fixed position and length within each row of the 'data' column.
# Ultimately, we will delete the 'data' column when it has all valuable information copied into other columns.

## Start with PIDN. This creates a PIDN column.
df['PIDN'] = np.where(df.data.str.match('Patient ID'), df.data.str.split(' ').str.get(2), None)
df['PIDN'] = df['PIDN'].fillna(method ="ffill")

# Visit
df['Visit'] = np.where(df.data.str.match('Patient ID'), df.data.str.split(' ').str.get(5), None)
df['Visit'] = df['Visit'].fillna(method ="ffill")

## ErrorCategory
df['ErrorCategory'] = None
df['ErrorCategory'] = np.where(df.data.str.match('Within Form Error Checks'), 'Within Form Error', df['ErrorCategory']) 
df['ErrorCategory'] = np.where(df.data.str.match('Cross Form Error Checks'), 'Cross Form Error', df['ErrorCategory']) 
df['ErrorCategory'] = np.where(df.data.str.match('Cross Visit'), 'Cross Visit', df['ErrorCategory'])
df['ErrorCategory'] = np.where(df.data.str.match('Range Checks'), 'Within Form Error Checks', df['ErrorCategory'])
df['ErrorCategory'] = np.where(df.data.str.match('Missing Form'), 'Cross Form Error', df['ErrorCategory'])
df['ErrorCategory'] = np.where(df.data.str.match('Packet Mismatch'), 'Cross Form Error', df['ErrorCategory']) 
df['ErrorCategory'] = df['ErrorCategory'].fillna(method ="ffill")

# ErrorType: start with Alerts. Shift them up by one since the NACC prints them after the alerts are shown
df['ErrorType'] = '' # creates an empty column
df['ErrorType'] = np.where(df.data.str.match('Verify this alert'), 'Alerts', df['ErrorType']) 
df.ErrorType = df.ErrorType.shift(-1)

df['ErrorType'] = np.where((df.ErrorCategory.str.match('Within') & df.ErrorType.str.match('(?!Alerts)')), 'Error Checks', df['ErrorType'])
df['ErrorType'] = np.where((df.ErrorCategory.str.match('Cross Form Error') & df.ErrorType.str.match('(?!Alerts)')), 'Error Checks', df['ErrorType']) 
df['ErrorType'] = np.where((df.ErrorCategory.str.match('Cross Visit') & df.ErrorType.str.match('(?!Alerts)')), 'Error Checks', df['ErrorType']) 

## ErrorType: find 'Found Not Missing'. Haven't tested this since our center doesn't have any on our error report.
df['ErrorType'] = np.where((df.data.str.match('Found Not Missing') & df.ErrorType.str.match('(?!Alerts)')), 'Found Not Missing', df['ErrorType'])

## ErrorType: find 'Verified And Not Approved'. Haven't tested this since our center doesn't have any on our error report.
df['ErrorType'] = np.where((df.data.str.match('Verified And Not Approved') & df.ErrorType.str.match('(?!Alerts)')), 'Error Checks', df['ErrorType'])

# Form column. This is a bit different than the previous strategy. Using regex, if `data` starts with 2 non-spaces and is followed by a space,
# we take that as the Form ID information, eg, 'B4 '.

# Here we also get the Forms that are missing for Missing Form(s) errors.
df['Form'] = ''
df.Form.loc[df[df.data.str.match('^\S{2}\s+')].index] = df.data.str.split(' ').str.get(0) # Matches UDS  forms that are 2 characters in length (e.g., 'C1')
df.Form.loc[df[df.data.str.match('^\S{3}\s+')].index] = df.data.str.split(' ').str.get(0) # Matches FTLD forms that are 3 characters in length (e.g., 'C2F')
df.Form.loc[df[df.data.str.match('Missing Form')].index] = df.data.str.replace('Missing Form\(s\): ', '')
df.Form.loc[df[df.data.str.match('F packet Version 3 must have a C1 or C2 form')].index] = 'C1 C2'
df.Form.loc[df[df.data.str.match('Version 3 Missing Form')].index] = 'C1 C2'
df.Form.loc[df[df.data.str.match('Form present indicated missing on Z1')].index] = df.data.str.split(' ').str.get(-1)

# Field column. Same regex as above; just grabbing the next character as the Field. Relying on the NACC having consistent fields in their error messages.
df['Field'] = '' 
df.Field.loc[df[df.data.str.match('^\S{2}\s+')].index] = df.data.str.split(' ').str.get(1) # Matches UDS  forms that are 2 characters in length
df.Field.loc[df[df.data.str.match('^\S{3}\s+')].index] = df.data.str.split(' ').str.get(1) # Matches FTLD forms that are 3 characters in length

# Varname column. 
df['Varname'] = ''
df.Varname.loc[df[df.data.str.match('^\S{2}\s+')].index] = df.data.str.split(' ').str.get(2) # as above for Field and Form
df.Varname.loc[df[df.data.str.match('^\S{3}\s+')].index] = df.data.str.split(' ').str.get(2) # as above for Field and Form
df.Varname.loc[df[df.data.str.match('Missing Form')].index] = ''            # Varname is not relevant for Missing Form(s) so set to blank
df.Varname.loc[df[df.data.str.match('Packet Mismatch')].index] = 'PACKET'   # This is a custom label made by us to help with error labeling...not from the NACC.

# ErrorCode column. Similar idea as above.
df['ErrorCode'] = ''
df.ErrorCode.loc[df[df.data.str.match('^\S{2}\s+')].index] = df.data.str.split(' ').str.get(3)
df.ErrorCode.loc[df[df.data.str.match('^\S{3}\s+')].index] = df.data.str.split(' ').str.get(3)
df.ErrorCode.loc[df[df.data.str.match('Missing Form')].index] = ''

## Packet. This was troublesome; the best solution was to do a string replace then split and str.get and ffill
df.data = df.data.str.replace('Packet\: FTLD Module Initial', ' IF ')
df.data = df.data.str.replace('Packet\: FTLD Module Followup', ' FF ')

df['Packet'] = np.where(df.data.str.match('Patient ID'), df.data.str.split(' ').str.get(7), None)


df['Packet'] = df['Packet'].fillna(method ="ffill")

# Remove rows no longer necessary from the original 'data' column that have been munged
df = df[df.data !='Range Checks']
df = df[df.data !='Item Data Element E-# Error Value']
df = df[df.data !='Item Data Element E-# Error Var Comp Value']
df = df[df.data !='Within Form Error Checks']
df = df[df.data !='Cross Form Alerts']
df = df[df.data !='Within Form Alerts']
df = df[df.data !='Verify this alert'] 
df = df[df.data !='Cross Form Error Checks']
df = df[df.data !='Item Data Element E-# Error Value Cross with']

# Time for ErrorText. This is the most involved part; there are many edgecases due to inconsistent formatting of the error messages by the NACC.
# Luckly, most NACC errors start with 4 fields, then comes the error text itself. 
df['ErrorText'] = None 

# First, fix an edge case: an error with 0 fields before the error message
df.data[df.data.str.startswith('Version 3 Missing Form(s): ')]= df.data.str.replace('Version 3 Missing Form\(s\)\:', '1 2 3 4 Version 3 Missing Form(s):') 

# If a NACC error doesn't have 4 fields before the actual message begins, prepend the text so that it fits this rule
desc1 = []
for astring in list(df.data):
   # print len(astring.split())
    if len(astring.split()) >=4:
        desc1.append(' '.join(astring.split()[4:]))
    else:
        desc1.append('1 2 3 4 5')
        
df['ErrorText'] = desc1
df.ErrorText.loc[df[df.data.str.match('Missing Form')].index] = df.data

# Some extra, one-off cleaning for odd situations, to make the ErrorText column cleaner
df.ErrorText = df.ErrorText.str.replace('Error: ', '') 
df.ErrorText = df.ErrorText.str.replace('Alert: ', '')
df.ErrorText = df.ErrorText.str.replace('mismatch', 'Packet mismatch: ') 

df.ErrorText = df.ErrorText.str.replace('on Z1\:', 'Form present indicated missing on Z1: ')
df.ErrorText = df.ErrorText.str.replace('on Z1F\:', 'Form present indicated missing on Z1F: ')
df.ErrorText = df.ErrorText.str.replace('Version 3 Missing Form(s): C1 C2', 'C1 C2')
df.ErrorText = df.ErrorText.str.replace('date after Module', 'Scan date after Module')
df.ErrorText = df.ErrorText.str.replace('must have a C1 or C2 form', 'F packet Version 3 must have a C1 or C2 form')

# Some errors need to have Var replaced at this point in the dataframe's creation
df.ErrorText = df.ErrorText.str.replace('must have a C1 or C2 form', 'F packet Version 3 must have a C1 or C2 form')
df.ErrorText = df.ErrorText.str.replace('have an etiologic diagnosis', 'Must have an etiologic diagnosis')
df.ErrorText = df.ErrorText.str.replace('a-e must be filled or empty', 'Items a-e must be filled or empty')


# To make it easier to read, change ErrorCategory values of 'Within Form Error Checks' or 'Within Form Checks' or 'Within Form Error' to 'Within Form',
# and change 'Cross Form Error' to 'Cross Form'
df.ErrorCategory = df.ErrorCategory.str.replace('Within Form Error', 'Within Form')
df.ErrorCategory = df.ErrorCategory.str.replace('Within Form Checks', 'Within Form')
df.ErrorCategory = df.ErrorCategory.str.replace('Within Form Error Checks', 'Within Form')
df.ErrorCategory = df.ErrorCategory.str.replace('Cross Form Error', 'Cross Form')

# Final removal of unwanted rows and the 'data' column itself
df = df[~df.data.str.match('^Patient ID')]
df = df.drop('data', axis=1)

# One-off cleanups of the error text
df.ErrorText[df.ErrorText.str.startswith('of')] = df.ErrorText.str.replace('of ', 'Value of ') 

# convert datatype in case you want to merge later on some other table you read into pandas
df.PIDN = df.PIDN.astype('int')

#uncomment to write a csv, don't forget the appropriate path
df.to_csv('error_report.csv', index='False')

In [2]:
df